<img align="right" style="padding-left:10px; height: 20%; width: 20%" src="figures/projector-300x300.jpg" ></a>

## Case Study: Movie Suggestion

### The Movies Dataset

Collaborative datasets for movies (and other products) can be large! Here is a [small (1 MB) subset of the IMDB database](https://grouplens.org/datasets/movielens/latest/), downloaded and unzipped for your convenience.

The dataset consists of 9742 movies.

In [ ]:
import numpy as np
from random import random, seed
seed(a=314159)
selector = [(random() < 0.2) for i in range(100836)]
# m = 40836
# n = 40836
m = sum([(1 if selector[i] else 0) for i in range(100836)])
n = m
print (m)
matrix = np.empty((m,n),dtype=np.float32)
matrix

In [3]:
import pandas as pd
import numpy as np
from pandas import DataFrame
movies_directory = '../../04-analysis-and-visualization/04-05-recommendations/ml-latest-small/'
movies = pd.read_csv(movies_directory+'movies.csv',header = 0) 

movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### Movie Ratings

The above 9742 movies were rated by 610 users; this works out to about 165 movies on average rated by each user, available in the `ratings.csv` file as sampled in the DataFrame below.

In [41]:
ratings = pd.read_csv(movies_directory+'ratings.csv',header = 0) 
ratings
from random import random, seed
seed(a=314159)
ratings = ratings.drop(ratings.index[[(random() < 0.85) for i in range(len(ratings.index))]])
# ratings[ratings.userId.isin([607])]
ratings

,userId,movieId,rating,timestamp
11,1,216,5.0,964981208
17,1,316,3.0,964982310
20,1,356,4.0,964980962
33,1,592,4.0,964982271
36,1,608,5.0,964982931
...,...,...,...,...
100818,610,159093,3.0,1493847704
100819,610,160080,3.0,1493848031
100827,610,163937,3.5,1493848789
100829,610,164179,5.0,1493845631


### Investigating the ratings dataset

Find the number of ratings created by each user.

In [42]:
# Ratings Group By userId:
# print (type(ratings.groupby(["userId"])["userId"].count())) # prints <class 'pandas.core.series.Series'>

# Convert Series to DataFrame
#     Ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.reset_index.html#pandas.Series.reset_index

counts = ratings.groupby(["userId"])["userId"].count().reset_index(name="Count")
counts

,userId,Count
0,1,35
1,2,1
2,3,9
3,4,31
4,5,3
...,...,...
602,606,143
603,607,32
604,608,115
605,609,7


<a href="https://en.wikipedia.org/wiki/Collaborative_filtering"><img align="right" style="padding-left:10px; height: 40%; width: 40%" src="https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif" ></a>

## General Approach

As discussed in [04-05-recommendations](../../04-analysis-and-visualization/04-05-recommendations/04-05-recommendations.ipynb) a generalized version of Collaborative filtering, implied by the adjoining image, is a three-step process:

1. A user expresses their preferences by rating items (e.g. books, movies or CDs) of the system. These ratings can be viewed as an approximate representation of the user's interest in the corresponding domain. _The ratings have been collected by IMDB and imported into the `ratings` DataFrame._
2. The system matches this user's ratings against other users' and finds the people with most "similar" tastes. For the purpose of this Case, we shall determine the recommendations for the user with **userId = 607**.
3. With similar users, the system recommends items that the similar users have rated highly but not yet being rated by this user (presumably the absence of rating is often considered as the unfamiliarity of an item).

<span style="color:blue">

### Solution Development
</span>

We proceed with the calculations as outlined above but first create a **tiny dataset** such that we can develop the solution and _verify the calculations manually._ 

Step 2 of the algorithms is for the system to match this user's ratings against other users' and finds the people with most "similar" tastes. We shall use **userId = 5** for this tiny dataset.

Once the solution has been developed, we will write functions and classes to package the developed code and use it for the given dataset.

In [43]:
# Cell 4
# Initial Parameters
given_userId = 607
threshold_distance = 3.5

In [44]:
# Cell 5
# Data Location
# Already initialized above, nothing to do!

In [45]:
# Cell 6
# Drop the timestamp column because we shall not be using it.
ratings = ratings.drop(columns=['timestamp'])
ratings

,userId,movieId,rating
11,1,216,5.0
17,1,316,3.0
20,1,356,4.0
33,1,592,4.0
36,1,608,5.0
...,...,...,...
100818,610,159093,3.0
100819,610,160080,3.0
100827,610,163937,3.5
100829,610,164179,5.0


### Librares to use

We will be using numpy and scipy for most of the calculations, mostly use `pandas` for pretty printing. 

Variable names will be chosen such that:

1. Variables ending in `_np` will be used for numpy arrays.
1. Variables ending in `_2d` will be used for 2-D numpy arrays.
2. Variables ending in `_df` will be used for pandas DataFrames.

In [46]:
# Cell 7
ratings_np = ratings.to_numpy(dtype=np.float32)
ratings_np

array([[1.00000e+00, 2.16000e+02, 5.00000e+00],
       [1.00000e+00, 3.16000e+02, 3.00000e+00],
       [1.00000e+00, 3.56000e+02, 4.00000e+00],
       ...,
       [6.10000e+02, 1.63937e+05, 3.50000e+00],
       [6.10000e+02, 1.64179e+05, 5.00000e+00],
       [6.10000e+02, 1.68248e+05, 5.00000e+00]], dtype=float32)

In [47]:
# Find the rating by our user
x = ratings_np
the_x_2d = x[np.where(x[:,0] == given_userId)][:, [2]]
the_x_2d = the_x_2d[0].reshape(1,1) # pick the first one in case we have multiple ratings records for user.
the_x = the_x_2d.reshape(the_x_2d.shape[0])
the_x

array([4.], dtype=float32)

In [48]:
# Find the ratings by all users
all_x_2d = ratings_np[:, [2]]
all_x = all_x_2d.reshape(all_x_2d.shape[0])
print (all_x)

all_u_2d = ratings_np[:, [0]]
all_u = all_u_2d.reshape(all_u_2d.shape[0])

all_m_2d = ratings_np[:, [1]]
all_m = all_m_2d.reshape(all_m_2d.shape[0])

all_u, all_m, all_x

[5.  3.  4.  ... 3.5 5.  5. ]


(array([  1.,   1.,   1., ..., 610., 610., 610.], dtype=float32),
 array([   216.,    316.,    356., ..., 163937., 164179., 168248.],
       dtype=float32),
 array([5. , 3. , 4. , ..., 3.5, 5. , 5. ], dtype=float32))

In [49]:
from scipy.spatial.distance import cdist, euclidean
from scipy.spatial import distance_matrix
dm = distance_matrix(all_x_2d, all_x_2d)

In [50]:
# Manually verifying the euclidean calculation.
# The numbers produced by the previous cell and by this cell should match!
dists = cdist(the_x_2d, all_x_2d)
distances = list(dists.reshape(dists.shape[1]))
# print (distances)

In [51]:
# This cell will throw an exception if the results of euclidean calculation don't match
from math import sqrt
assert(np.isclose(euclidean(the_x, all_x), 
                  sqrt(sum([_*_ for _ in distances]))))

In [53]:
# Cell 13
dists = np.stack([all_u,
                  all_m,
                  np.apply_along_axis(lambda x: sqrt(sum([_*_ for _ in x])), 0, dm)])
dists_df = DataFrame(dists.transpose(), columns=['u', 'm', 'x']) \
           .astype({'u': 'int32', 'm': 'int32', 'x': 'float32'})
dists_df   #  [['u', 'x']]

,u,m,x
0,1,216,226.368057
1,1,316,143.103104
2,1,356,143.848877
3,1,592,143.848877
4,1,608,226.368057
...,...,...,...
15163,610,159093,143.103104
15164,610,160080,143.103104
15165,610,163937,129.589737
15166,610,164179,226.368057


In [54]:
dists_df = dists_df.sort_values(by ='x' , ascending=True)
dists_df

,u,m,x
9666,414,8376,129.589737
14751,606,4313,129.589737
11047,474,2184,129.589737
14750,606,4054,129.589737
11044,474,2176,129.589737
...,...,...,...
6775,298,88785,390.721771
11547,480,8807,390.721771
13255,567,108190,390.721771
14856,608,181,390.721771


In [58]:
# Cell 14
# Filtering out the users that are not like our user.
threshold_distance = 130
similar_users_df = dists_df[dists_df['x'] < threshold_distance]
similar_users_df

,u,m,x
9666,414,8376,129.589737
14751,606,4313,129.589737
11047,474,2184,129.589737
14750,606,4054,129.589737
11044,474,2176,129.589737
...,...,...,...
11740,484,3825,129.589737
7708,332,2395,129.589737
7582,328,2080,129.589737
548,21,103883,129.589737


### Recommending the movies our user may want to watch

In **step 3 of the algorithm**, the system recommends items that the similar users have rated highly but not yet being rated by this user (presumably the absence of rating is often considered as the unfamiliarity of an item).

In [56]:
# Cell 15
# What movies has the user picked already? Don't recommend those!
candidates_df = dists_df[dists_df['u'] == 607]
candidates_np = candidates_df['m']
dists_np = dists_df['m']
candidate_movies = [i for i in set(dists_np) if not i in set(candidates_np) ]
#print (candidate_movies)


In [57]:
# Candidate Movies
np.isin(movies['movieId'], list(candidate_movies))
movies[np.isin(movies['movieId'], list(candidate_movies))]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9714,188675,Dogman (2018),Crime|Drama
9720,189333,Mission: Impossible - Fallout (2018),Action|Adventure|Thriller
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy


## TO-DO

Your assignment is to package the code from <span style="color:green"><em># Initial Parameters</em></span> to the <span style="color:green"><em># Candidate Movies</em></span> cells.

1. Create a function `recommend_movies(uid,threshold)` that takes `userId`, `threshold` and `movies_directory` as parameters and produces recommendations for the user. Test the code first with userId = 607. Try various values of threshold such that the user gets at least 6 movie recommendations.
2. Back in the <span style="color:green"><em># What movies has the user picked already? Don't recommend those!</em></span> cell (cell 15), we had picked the first record for our user. Modify the code to begin instead with the movie our user liked the most!
3. Time your code for various values of `userId` and `threshold`. What accounts for the variation in timing?